In [ ]:
# A proof of concept showing using a genetic algorithm with our environment.
# It is similar to https://github.com/DEAP/deap/blob/a0b78956e28387785e3bb6e2b4b1f1b32c2b3883/examples/ga/onemax_short.py

import array
import random

import numpy as np

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import gym
from environments.SimpleEnv import SimpleEnv

from ray.rllib.agents import ppo
from ray import tune
from ray.rllib.policy.policy import Policy
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import ray

ray.init()


creator.create('FitnessMax', base.Fitness, weights=(1.0, ))
creator.create('Individual', array.array, typecode='d',
               fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register('attr', random.uniform, -1, 1)
toolbox.register('individual', tools.initRepeat, creator.Individual,
                 toolbox.attr, 2)
toolbox.register('population', tools.initRepeat, list,
                 toolbox.individual)


def evaluate(individual, pop):
    """Runs the environment. Selects random agents from pop.
  It returns the total true reward as the fitness.
  TODO: Assumes n_agents is 2.
  TODO: This is very slow, can we train the whole population at the same time?
  """
    
    #Select random individuals from pop and create the reward weights
    individual = np.asarray(individual)
    reward_weights = [individual, np.random.default_rng().choice(pop, 1, replace=False)[0]]
    reward_weights = np.asarray(reward_weights)
    #print(individual, reward_weights)
    
    #env is only to get action space and observation space
    env = SimpleEnv(config={
        'n_agents': 2,
        'n_vars': 2,
        'reward_weights': reward_weights,
        'max_step_count': 10,
    })
    
    #Callback functions to keep track of true reward while training
    def on_episode_start(info):
        episode = info["episode"]
        episode.user_data["true_rewards"] = []

    def on_episode_step(info):
        #true reward is stored in the env object. Would be more elegant
        #to return it in the info dict of step, but could not get last_info_for to work
        episode = info["episode"]
        env = info["env"]
        true_reward = env.env_states[0].env.last_true_reward
        episode.user_data["true_rewards"].append(true_reward)

    def on_episode_end(info):
        episode = info["episode"]
        true_reward = np.sum(episode.user_data["true_rewards"])
        episode.custom_metrics["true_reward"] = true_reward
        
    trainer = ppo.PPOTrainer(env=SimpleEnv, config={
        "multiagent": {
            "policies": {
                # the first tuple value is None -> uses default policy
                "agent_0": (None, env.observation_space, env.action_space, {}),
                "agent_1": (None, env.observation_space, env.action_space, {}),
            },
            "policy_mapping_fn":
                lambda agent_id:
                    agent_id
        },
        'env_config': {
            'n_agents': 2,
            'n_vars': 2,
            'reward_weights': reward_weights,
            'max_step_count': 10,
        },
        "callbacks": {
            "on_episode_start": on_episode_start,
            "on_episode_step": on_episode_step,
            "on_episode_end": on_episode_end,
        },
    })
    
    for i in range(1):
        print('TRAINING', i)
        trainer.train()  # distributed training step
        
    print('true reward', trainer.collect_metrics()['custom_metrics']['true_reward_mean'])
    return (trainer.collect_metrics()['custom_metrics']['true_reward_mean'], )


toolbox.register('evaluate', evaluate)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutFlipBit, indpb=0.05)
toolbox.register('select', tools.selTournament, tournsize=3)

def evolve(population, toolbox, cxpb, mutpb, ngen, stats=None, 
           halloffame=None, verbose=__debug__):
    """
    Almost identical to deap.algorithms.eaSimple. 
    
    TODO: Sometimes all individuals become identical for some reason
    Runtime is O(pop size * ngen * RL iterations) with a huge constant. 
    Easily parralizable, but we can remove the pop size factor by training the
    entire population at once. Requires environments to support many agents.
    """
    
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    #print('population1', population)
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind, [invalid_ind] * len(invalid_ind))
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    #print('population2', population)
    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        print('population', population)
        offspring = toolbox.select(population, len(population))
        #print('offspring', offspring)

        # Vary the pool of individuals
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)
        #print('offspring1', offspring)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        #print('invalid_ind', invalid_ind)
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind, [invalid_ind] * len(invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

def main():
    pop = toolbox.population(n=10)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register('avg', np.mean)
    stats.register('std', np.std)
    stats.register('min', np.min)
    stats.register('max', np.max)

    (pop, log) = evolve(
        pop,
        toolbox,
        cxpb=0.5,
        mutpb=0.2,
        ngen=10,
        stats=stats,
        halloffame=hof,
        verbose=True,
        )

    print ('pop', pop)

    return (pop, log, hof)


if __name__ == '__main__':
    main()


/home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2020-04-30 22:04:12,009	INFO resource_spec.py:212 -- Starting Ray with 3.47 GiB memory available for workers and up to 1.76 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-30 22:04:12,476	INFO services.py:1148 -- View the Ray dashboard at localhost:8265
2020-04-30 22:04:13,128	INFO trainer.py:428 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-04-30 22:04:13,192	ERROR syncer.py:39 -- Log sync requires rsync to be installed.
2020-04-30 22:04:13,195	INFO trainer.py:585 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -

(pid=2566) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=2566)   from ._conv import register_converters as _register_converters
(pid=2565) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=2565)   from ._conv import register_converters as _register_converters


2020-04-30 22:04:23,380	INFO trainable.py:180 -- _setup took 10.187 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2020-04-30 22:04:23,381	INFO trainable.py:217 -- Getting current IP.
2020-04-30 22:04:23,381	WARNING util.py:37 -- Install gputil for GPU system monitoring.


TRAINING 0


2020-04-30 22:04:38,762	ERROR syncer.py:39 -- Log sync requires rsync to be installed.


...

TRAINING 0


2020-04-30 22:07:35,312	ERROR syncer.py:39 -- Log sync requires rsync to be installed.


true reward -0.09613980476395227
(pid=3444) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=3444)   from ._conv import register_converters as _register_converters
(pid=3443) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=3443)   from ._conv import register_converters as _register_converters


2020-04-30 22:07:45,117	INFO trainable.py:217 -- Getting current IP.
2020-04-30 22:07:45,118	WARNING util.py:37 -- Install gputil for GPU system monitoring.


TRAINING 0


In [ ]:
#I had problems with rllib running out of memory after running many tests. Havn't found a solution yet
ray.shutdown()